# Chain-of-Agents Part 2: Progressive Filtering - Quality Control at Scale

**Time**: 45 minutes | **Level**: Intermediate | **Prerequisite**: Part 1

## The Problem We're Solving

You have 10,000 trajectories from multi-agent systems. But:
- Some are garbage (agents went off-topic)
- Some are redundant (same solution 100 times)
- Some are gold (perfect agent collaboration)

**Bad training data = Bad AFM**

Progressive filtering finds the gold. Let's build it from scratch.

## Step 1: Generate Realistic Trajectories (Good and Bad)

First, let's create trajectories with varying quality.

In [ ]:
import random
import json

def generate_trajectory(task, quality="good"):
    """Generate trajectories of different quality levels"""
    
    if quality == "good":
        # Good trajectory: clear, complete, collaborative
        trajectory = [
            {"agent": "Planner", "output": f"Breaking down '{task}' into 3 clear steps..."},
            {"agent": "Coder", "output": f"Implementing solution with proper error handling..."},
            {"agent": "Reviewer", "output": f"Code reviewed. Tests pass. Ready for production."}
        ]
    elif quality == "bad":
        # Bad trajectory: incomplete, errors, off-topic
        trajectory = [
            {"agent": "Planner", "output": f"I don't understand '{task}'..."},
            {"agent": "Coder", "output": f"Error: undefined variable..."},
            {"agent": "Reviewer", "output": f"This doesn't work."}
        ]
    else:  # medium
        # Medium trajectory: okay but not great
        trajectory = [
            {"agent": "Planner", "output": f"Working on '{task}'..."},
            {"agent": "Coder", "output": f"Basic implementation done."},
            {"agent": "Reviewer", "output": f"Looks okay."}
        ]
    
    return {
        "task": task,
        "trajectory": trajectory,
        "quality": quality  # Ground truth for testing
    }

# Generate a mix of trajectories
tasks = ["Build API", "Fix bug", "Add feature", "Write tests", "Deploy app"]
trajectories = []

for _ in range(30):  # Generate 30 trajectories
    task = random.choice(tasks)
    quality = random.choice(["good", "good", "medium", "bad"])  # 50% good, 25% medium, 25% bad
    trajectories.append(generate_trajectory(task, quality))

# Show distribution
quality_counts = {}
for t in trajectories:
    q = t['quality']
    quality_counts[q] = quality_counts.get(q, 0) + 1

print("📊 Generated Trajectory Quality Distribution:")
for quality, count in quality_counts.items():
    bar = '█' * (count // 2)
    print(f"  {quality:7} {bar:15} {count} trajectories")
print(f"\nTotal: {len(trajectories)} trajectories")

## Step 2: Build Quality Metrics

What makes a trajectory good for training an AFM?

In [ ]:
def calculate_trajectory_metrics(trajectory_data):
    """Calculate quality metrics for a trajectory"""
    
    trajectory = trajectory_data['trajectory']
    metrics = {}
    
    # Metric 1: Total output length (longer usually = more detailed)
    total_length = sum(len(step['output']) for step in trajectory)
    metrics['output_length'] = total_length
    
    # Metric 2: Agent diversity (all agents should contribute)
    unique_agents = len(set(step['agent'] for step in trajectory))
    metrics['agent_diversity'] = unique_agents / len(trajectory) if trajectory else 0
    
    # Metric 3: Completion indicators (look for success signals)
    final_output = trajectory[-1]['output'].lower() if trajectory else ""
    success_words = ['complete', 'success', 'ready', 'pass', 'done', 'works']
    metrics['has_completion'] = any(word in final_output for word in success_words)
    
    # Metric 4: Error indicators (look for failure signals)
    all_outputs = ' '.join(step['output'].lower() for step in trajectory)
    error_words = ['error', 'fail', 'undefined', "doesn't work", "don't understand"]
    metrics['has_errors'] = any(word in all_outputs for word in error_words)
    
    # Metric 5: Progressive depth (each step should build on previous)
    if len(trajectory) > 1:
        lengths = [len(step['output']) for step in trajectory]
        metrics['is_progressive'] = lengths[-1] >= lengths[0]  # Last step more detailed
    else:
        metrics['is_progressive'] = False
    
    return metrics

# Test on one trajectory
example = trajectories[0]
metrics = calculate_trajectory_metrics(example)

print(f"📏 Metrics for trajectory (quality={example['quality']}):")
print(json.dumps(metrics, indent=2))

## Step 3: Simple Scoring Function

Combine metrics into a single quality score.

In [ ]:
def score_trajectory(trajectory_data):
    """Score trajectory from 0-100 based on quality metrics"""
    
    metrics = calculate_trajectory_metrics(trajectory_data)
    score = 50  # Start at neutral
    
    # Positive factors
    if metrics['output_length'] > 100:
        score += 10
    if metrics['output_length'] > 200:
        score += 10
    if metrics['agent_diversity'] > 0.8:
        score += 15
    if metrics['has_completion']:
        score += 20
    if metrics['is_progressive']:
        score += 10
    
    # Negative factors
    if metrics['has_errors']:
        score -= 30
    if metrics['output_length'] < 50:
        score -= 20
    if metrics['agent_diversity'] < 0.5:
        score -= 15
    
    # Clamp to 0-100
    return max(0, min(100, score))

# Score all trajectories
for t in trajectories:
    t['score'] = score_trajectory(t)

# Show score distribution by quality
print("📊 Average Scores by Quality:")
for quality in ['good', 'medium', 'bad']:
    scores = [t['score'] for t in trajectories if t['quality'] == quality]
    if scores:
        avg_score = sum(scores) / len(scores)
        print(f"  {quality:7} → {avg_score:.1f}/100")

## Step 4: Progressive Filtering Pipeline

Now the magic: filter in stages, getting stricter each time.

In [ ]:
def progressive_filter(trajectories, stages=3):
    """Apply progressive filtering to trajectories"""
    
    filtered = trajectories.copy()
    
    print("🔄 PROGRESSIVE FILTERING PIPELINE")
    print("="*50)
    print(f"Starting with {len(filtered)} trajectories\n")
    
    # Define filtering thresholds for each stage
    thresholds = [30, 50, 70]  # Increasingly strict
    
    for stage, threshold in enumerate(thresholds[:stages], 1):
        print(f"📍 Stage {stage}: Score threshold >= {threshold}")
        
        # Filter based on score
        before_count = len(filtered)
        filtered = [t for t in filtered if t['score'] >= threshold]
        after_count = len(filtered)
        
        # Show what happened
        removed = before_count - after_count
        removal_rate = (removed / before_count * 100) if before_count > 0 else 0
        
        print(f"  Removed: {removed} ({removal_rate:.1f}%)")
        print(f"  Remaining: {after_count}")
        
        # Show quality distribution at this stage
        quality_dist = {}
        for t in filtered:
            q = t['quality']
            quality_dist[q] = quality_dist.get(q, 0) + 1
        print(f"  Quality: {quality_dist}")
        print()
    
    print("="*50)
    print(f"✅ Final: {len(filtered)}/{len(trajectories)} trajectories kept")
    print(f"📈 Quality improvement: {(len(filtered)/len(trajectories)*100):.1f}% survival rate")
    
    return filtered

# Apply progressive filtering
filtered_trajectories = progressive_filter(trajectories)

## Step 5: Visualize Filtering Effects

Let's see how filtering improves quality distribution.

In [ ]:
def visualize_filtering_effect(original, filtered):
    """Visualize the effect of filtering on quality distribution"""
    
    print("\n📊 FILTERING EFFECT VISUALIZATION\n" + "="*50)
    
    # Calculate distributions
    def get_distribution(trajectory_list):
        dist = {'good': 0, 'medium': 0, 'bad': 0}
        for t in trajectory_list:
            dist[t['quality']] += 1
        return dist
    
    orig_dist = get_distribution(original)
    filt_dist = get_distribution(filtered)
    
    # Display side by side
    print("BEFORE FILTERING:")
    for quality in ['good', 'medium', 'bad']:
        count = orig_dist[quality]
        pct = (count / len(original) * 100) if original else 0
        bar = '█' * int(pct / 2)
        print(f"  {quality:7} {bar:25} {count:2} ({pct:.1f}%)")
    
    print("\nAFTER FILTERING:")
    for quality in ['good', 'medium', 'bad']:
        count = filt_dist[quality]
        pct = (count / len(filtered) * 100) if filtered else 0
        bar = '█' * int(pct / 2)
        print(f"  {quality:7} {bar:25} {count:2} ({pct:.1f}%)")
    
    # Calculate improvement
    if filtered:
        good_before = orig_dist['good'] / len(original) * 100
        good_after = filt_dist['good'] / len(filtered) * 100
        improvement = good_after - good_before
        
        print(f"\n✨ Quality Improvement:")
        print(f"  Good trajectories: {good_before:.1f}% → {good_after:.1f}%")
        print(f"  Improvement: +{improvement:.1f}% good trajectories")

visualize_filtering_effect(trajectories, filtered_trajectories)

## Step 6: Advanced Filtering - Diversity & Deduplication

Good trajectories aren't enough. We need DIVERSE good trajectories.

In [ ]:
def calculate_similarity(traj1, traj2):
    """Calculate similarity between two trajectories (0-1)"""
    
    # Simple approach: compare outputs
    outputs1 = ' '.join(s['output'] for s in traj1['trajectory'])
    outputs2 = ' '.join(s['output'] for s in traj2['trajectory'])
    
    # Character-level similarity (simplified)
    common_chars = sum(1 for c1, c2 in zip(outputs1[:100], outputs2[:100]) if c1 == c2)
    max_len = max(len(outputs1[:100]), len(outputs2[:100]))
    
    return common_chars / max_len if max_len > 0 else 0

def diversity_filter(trajectories, min_diversity=0.3):
    """Keep only diverse trajectories (remove near-duplicates)"""
    
    print("\n🌈 DIVERSITY FILTERING")
    print("="*50)
    
    diverse_set = []
    
    for i, traj in enumerate(trajectories):
        # Check similarity with existing diverse set
        is_diverse = True
        
        for existing in diverse_set:
            similarity = calculate_similarity(traj, existing)
            if similarity > (1 - min_diversity):  # Too similar
                is_diverse = False
                break
        
        if is_diverse:
            diverse_set.append(traj)
            print(f"  ✓ Added trajectory {i+1} (unique)")
        else:
            print(f"  ✗ Skipped trajectory {i+1} (too similar)")
    
    print(f"\nResult: {len(diverse_set)}/{len(trajectories)} diverse trajectories")
    return diverse_set

# Apply diversity filtering
diverse_trajectories = diversity_filter(filtered_trajectories[:10])  # Demo on first 10

## Step 7: The Complete Pipeline

Put it all together: score → filter → diversify.

In [ ]:
class TrajectoryFilter:
    """Complete progressive filtering pipeline for CoA"""
    
    def __init__(self, min_score=50, diversity_threshold=0.3):
        self.min_score = min_score
        self.diversity_threshold = diversity_threshold
        self.stats = {}
    
    def process(self, trajectories):
        """Apply complete filtering pipeline"""
        
        print("🚀 COMPLETE FILTERING PIPELINE")
        print("="*60)
        
        # Step 1: Calculate scores
        print("\n1️⃣ Scoring trajectories...")
        for t in trajectories:
            t['score'] = score_trajectory(t)
        avg_score = sum(t['score'] for t in trajectories) / len(trajectories)
        print(f"   Average score: {avg_score:.1f}")
        
        # Step 2: Quality filtering
        print("\n2️⃣ Quality filtering...")
        quality_filtered = [t for t in trajectories if t['score'] >= self.min_score]
        print(f"   Kept: {len(quality_filtered)}/{len(trajectories)}")
        
        # Step 3: Diversity filtering
        print("\n3️⃣ Diversity filtering...")
        final_set = []
        for t in quality_filtered:
            is_unique = True
            for existing in final_set:
                if calculate_similarity(t, existing) > (1 - self.diversity_threshold):
                    is_unique = False
                    break
            if is_unique:
                final_set.append(t)
        print(f"   Kept: {len(final_set)}/{len(quality_filtered)}")
        
        # Step 4: Final statistics
        print("\n4️⃣ Final statistics:")
        self.stats = {
            'input_count': len(trajectories),
            'output_count': len(final_set),
            'retention_rate': len(final_set) / len(trajectories) * 100,
            'avg_score_before': avg_score,
            'avg_score_after': sum(t['score'] for t in final_set) / len(final_set) if final_set else 0
        }
        
        for key, value in self.stats.items():
            if 'count' in key:
                print(f"   {key}: {value}")
            else:
                print(f"   {key}: {value:.1f}")
        
        print("\n" + "="*60)
        print(f"✅ Filtering complete: {len(final_set)} high-quality trajectories")
        
        return final_set

# Run the complete pipeline
filter_pipeline = TrajectoryFilter(min_score=50, diversity_threshold=0.3)
final_trajectories = filter_pipeline.process(trajectories)

## Step 8: Why This Matters for AFM Performance

Let's simulate how filtering affects final model performance.

In [ ]:
def simulate_afm_performance(training_trajectories):
    """Simulate AFM performance based on training data quality"""
    
    # Calculate quality metrics
    quality_scores = [t['score'] for t in training_trajectories]
    avg_quality = sum(quality_scores) / len(quality_scores) if quality_scores else 0
    
    # Count quality distribution
    good_count = sum(1 for t in training_trajectories if t['quality'] == 'good')
    bad_count = sum(1 for t in training_trajectories if t['quality'] == 'bad')
    
    # Simulate performance (simplified model)
    base_performance = 0.40  # Base GAIA score
    
    # Good trajectories improve performance
    performance_boost = (good_count / len(training_trajectories)) * 0.15 if training_trajectories else 0
    
    # Bad trajectories hurt performance
    performance_penalty = (bad_count / len(training_trajectories)) * 0.10 if training_trajectories else 0
    
    # Quality bonus
    quality_bonus = (avg_quality / 100) * 0.05
    
    final_performance = base_performance + performance_boost - performance_penalty + quality_bonus
    
    return min(final_performance, 0.60)  # Cap at 60% for realism

# Compare performance with and without filtering
print("🎯 AFM PERFORMANCE SIMULATION\n" + "="*50)

# Without filtering
perf_unfiltered = simulate_afm_performance(trajectories)
print(f"Without filtering:")
print(f"  Training size: {len(trajectories)} trajectories")
print(f"  GAIA score: {perf_unfiltered:.3f} ({perf_unfiltered*100:.1f}%)")

# With filtering
perf_filtered = simulate_afm_performance(final_trajectories)
print(f"\nWith progressive filtering:")
print(f"  Training size: {len(final_trajectories)} trajectories")
print(f"  GAIA score: {perf_filtered:.3f} ({perf_filtered*100:.1f}%)")

# Show improvement
improvement = (perf_filtered - perf_unfiltered) / perf_unfiltered * 100
print(f"\n📈 Improvement: +{improvement:.1f}% relative performance")
print(f"\n💡 This is why CoA achieves 55.3% on GAIA!")
print(f"   Quality data > Quantity of data")

## Exercise 1: Build Your Own Quality Metric 🎯

Can you create a better quality metric?

In [ ]:
def your_quality_metric(trajectory_data):
    """Design your own trajectory quality metric"""
    
    # TODO: Implement your metric
    # Ideas to try:
    # - Semantic coherence between steps
    # - Increasing complexity across agents
    # - Task completion indicators
    # - Code quality signals (for coding tasks)
    # - Proper handoffs between agents
    
    score = 50  # Your implementation here
    
    return score

# Test your metric
print("Testing your metric:")
for t in trajectories[:3]:
    your_score = your_quality_metric(t)
    original_score = t['score']
    print(f"Task: {t['task']}, Quality: {t['quality']}")
    print(f"  Original score: {original_score:.1f}")
    print(f"  Your score: {your_score:.1f}")
    print()

## Exercise 2: Adaptive Filtering 🔄

Make filtering adapt based on data distribution.

In [ ]:
def adaptive_filter(trajectories, target_count=10):
    """Adaptively filter to get exactly target_count best trajectories"""
    
    # TODO: Implement adaptive filtering
    # Requirements:
    # 1. Always return exactly target_count trajectories
    # 2. Maximize average quality
    # 3. Maintain diversity
    # 4. Adjust thresholds automatically
    
    # Your implementation here
    filtered = trajectories[:target_count]  # Placeholder
    
    return filtered

# Test adaptive filtering
result = adaptive_filter(trajectories, target_count=5)
print(f"Adaptive filter returned {len(result)} trajectories")
print(f"Target was 5: {'✅ PASS' if len(result) == 5 else '❌ FAIL'}")

## Exercise 3: Filter Efficiency Challenge ⚡

Can you filter 10,000 trajectories in under 1 second?

In [ ]:
import time

def fast_filter(trajectories):
    """Ultra-fast filtering for large-scale datasets"""
    
    # TODO: Implement fast filtering
    # Hints:
    # - Pre-compute metrics
    # - Use vectorized operations
    # - Early stopping
    # - Approximate algorithms
    
    # Your implementation here
    filtered = trajectories  # Placeholder
    
    return filtered

# Generate large dataset
large_dataset = trajectories * 100  # 3000 trajectories

# Time your implementation
start = time.time()
result = fast_filter(large_dataset)
elapsed = time.time() - start

print(f"Filtered {len(large_dataset)} → {len(result)} trajectories")
print(f"Time: {elapsed:.3f} seconds")
print(f"Speed: {len(large_dataset)/elapsed:.0f} trajectories/second")
print(f"Target: < 1 second for 10,000: {'✅ FAST' if elapsed < 1 else '❌ TOO SLOW'}")

## Key Takeaways 🎓

1. **Quality > Quantity**: 10 good trajectories beat 1000 bad ones
2. **Progressive filtering**: Start loose, get stricter gradually
3. **Diversity matters**: Similar trajectories don't add value
4. **Metrics are key**: Good metrics = good filtering
5. **Performance impact**: Filtering can improve GAIA scores by 10-15%!

## The CoA Secret Sauce 🌟

Progressive filtering is WHY Chain-of-Agents works:
- **Traditional**: Train on everything → mediocre model
- **CoA**: Train on filtered gold → state-of-the-art model

## What's Next?

Part 3: **SFT - Distilling into AFM**. We'll take these filtered trajectories and train a single model that can simulate ALL agents!

## Homework 📝

1. Filter a dataset of 10,000+ trajectories
2. Experiment with different quality metrics
3. Plot quality distribution before/after filtering
4. Implement semantic similarity (not just character-level)
5. Read the CoA paper section on progressive filtering

Remember: **Good data is the foundation of good AI!** 🏗️